   - 데이터셋: Fashion MNIST

   - 미니배치 사이즈: 10, 100, 1000

   - 에폭 수: 30

   - 최적화 알고리즘: SGD, Adam 

   - 과적합 방지 알고리즘: 사용X, Batch Normalization, Weight Decay

   - 18개 설정(미니배치3X최적화2X과적합방지3)에 대해 모두 학습하고 학습 정확도 및 시험 정확도를 수치와 그래프로 표시하되 시험정확도가 높은 순으로 표시하세요.

   - 추가로, 가장 시험정확도가 높은 모델을 이용해 시험데이터 중 샘플로 5개를 읽어들여 그림을 표시하고 분류결과를 표시하여 확인하는 코드를 작성하세요. 

# 목차
- [Config.json 활용](#Batch_size,-Overffing-prevention,-optimizer-의-입력값-설정)
- [Create Model](#모델-경우-모두-생성)

#### Batch_size, Overffing prevention, optimizer 의 입력값 설정

   - 미니배치 사이즈: 10, 100, 1000

   - 에폭 수: 30 (고정)

   - 최적화 알고리즘: SGD, Adam 

   - 과적합 방지 알고리즘: 사용X, Batch Normalization, Weight Decay
   
   총 18가지 경우의 수가 존재
<hr>

In [3]:
with open("./config.json","r") as f:
    conf = json.load(f)
conf

{'option': {'epochs': 30,
  'batch_size': [10, 100, 1000],
  'overfitting': ['None', 'Batch Normalization', 'Weigth Decay'],
  'optimizers': ['Adam', 'SGD']}}

#### 모델 경우 모두 생성

In [11]:
count = 1
for batch in conf["option"]["batch_size"]:
    for overfit in conf["option"]["overfitting"]
    globals()['model%d'count] = 
    

SyntaxError: cannot assign to operator (Temp/ipykernel_5520/1487228169.py, line 2)

In [1]:
import json
import numpy as np
import mnist_reader
from Trainer import Trainer

In [2]:
X_train, y_train = mnist_reader.load_mnist('../data/fashion', kind='train')
X_test, y_test = mnist_reader.load_mnist("../data/fashion", kind="t10k")

In [6]:
model = Trainer("none", X_train, y_train, X_test, y_test, mini_batch_size=10, optimizer="SGD")

In [7]:
print(model.network)

In [8]:
model.train()

=== epoch:1, train acc:0.1, test acc:0.1 ===
=== epoch:2, train acc:0.3905, test acc:0.3898 ===
=== epoch:3, train acc:0.36943333333333334, test acc:0.3678 ===
=== epoch:4, train acc:0.28973333333333334, test acc:0.2879 ===
=== epoch:5, train acc:0.09998333333333333, test acc:0.1 ===


KeyboardInterrupt: 

In [9]:
class Trainer:
    def __init__(self, network, x_train, t_train, x_test, t_test,
                 epochs=30, mini_batch_size=10,  #default 10
                 optimizer="SGD", optimizer_param={'lr':0.01}, verbose=True):
        
        self.verbose = verbose
        self.x_train = x_train
        self.t_train = t_train
        self.x_test = x_test
        self.t_test = t_test
        self.epochs = epochs
        self.batch_size = mini_batch_size

        # optimzer
        optimizer_class_dict = {'sgd':SGD,'adam':Adam}
        model_class_dict = {"none": ThreeLayerNet, "batchnormalization" : ThreeLayerNetExtended, "weigthdecay" : ThreeLayerNetExtendedWD}
        self.network = model_class_dict[network.lower()](784, 100, 10)
        print(self.network.input_size)
        self.optimizer = optimizer_class_dict[optimizer.lower()](**optimizer_param)
        
        self.train_size = x_train.shape[0]
        self.iter_per_epoch = max(self.train_size / mini_batch_size, 1)
        self.max_iter = int(epochs * self.iter_per_epoch)
        self.current_iter = 0
        self.current_epoch = 0
        
        self.train_loss_list = []
        self.train_acc_list = []
        self.test_acc_list = []

    def train_step(self):
        batch_mask = np.random.choice(self.train_size, self.batch_size)
        x_batch = self.x_train[batch_mask]
        t_batch = self.t_train[batch_mask]
        
        grads = self.network.gradient(x_batch, t_batch)
        self.optimizer.update(self.network.params, grads)
        
        loss = self.network.loss(x_batch, t_batch)
        self.train_loss_list.append(loss)
        
        #if self.verbose: 
         #   print("train loss:" + str(loss))
        
        if self.current_iter % self.iter_per_epoch == 0:
            self.current_epoch += 1
                            
            train_acc = self.network.accuracy(self.x_train, self.t_train)
            test_acc = self.network.accuracy(self.x_test, self.t_test)
            
            self.train_acc_list.append(train_acc)
            self.test_acc_list.append(test_acc)

            if self.verbose: 
                print("=== epoch:" + str(self.current_epoch) + ", train acc:" + str(train_acc) + ", test acc:" + str(test_acc) + " ===")

        self.current_iter += 1

    def train(self):
        for i in range(self.max_iter):
            self.train_step()

        test_acc = self.network.accuracy(self.x_test, self.t_test)

        if self.verbose:
            print("=============== Final Test Accuracy ===============")
            print("test acc:" + str(test_acc))